# YOLOv8目标检测使用范例——分布式光伏实例分割

In [1]:
import sys
from osgeo import gdal
from ultralytics import YOLO
import cv2
import numpy as np
import time
import random
import yaml
import json
import logging
import os
from shapely import Polygon, MultiPolygon
from tqdm import tqdm
import requests
sys.path.append('../../ultralytics/dataset_preparation')
sys.path.append('../../ultralytics/inference/utils')
import yolo_dataset_utils
import split_image_v1
import split_image_v2
import time
import yaml
import json
import yolov8_seg_handle
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.gif', '.webp')


## 1.数据处理
### 下载标注数据集
标注数据集已经是yolo格式了。

In [ ]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download&confirm=1"

    session = requests.Session()

    response = session.get(URL, params={"id": id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {"id": id, "confirm": token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith("download_warning"):
            return value

    return None


def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

id = "1tvEtQ_GXn_5VuQqO2rj9DChrQ8iBF1pz"
destination = "solar_panel.zip"
download_file_from_google_drive(id, destination)
import zipfile
import os

zip_path = "solar_panel.zip"
extract_path = "./solar_panel"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for file_info in zip_ref.infolist():
        file_name = file_info.filename

        # Attempt to decode the file name
        try:
            file_name = file_name.encode('cp437').decode('gbk')  # Adjust the encoding as needed
        except:
            file_name = file_name.encode('cp437').decode('utf-8', 'ignore')  # Fallback to UTF-8

        extracted_path = os.path.join(extract_path, file_name)
        
        # Create directory structure
        if file_info.is_dir():
            os.makedirs(extracted_path, exist_ok=True)
        else:
            os.makedirs(os.path.dirname(extracted_path), exist_ok=True)
            
            # Extract and write the file
            with open(extracted_path, 'wb') as f:
                f.write(zip_ref.read(file_info.filename))



运行完成后，solar_panel下会有两个文件夹：images和label_txt。现在图片都是原始的tif文件，太大，不利于模型训练，要将tif图切割成小块，方便训练

运行以下代码，划分训练集、测试集与验证集。这里由于数据集较小，只划分训练集和验证集，比例为9:1

In [ ]:
img_path = 'solar_panel/images'
txt_path = 'solar_panel/label_txt'
# 运行以下脚本，划分训练集、验证集和测试集
yolo_dataset_utils.split_train_val_test(img_path, txt_path, val_percentage=0.1, test_percentage=0)

运行以下代码，将大图按照一定比例分割成小图

In [4]:
# 分别进入train, val, test目录下，运行分割图的脚本。运行后手动删除除了images和labels的其余文件夹。
split_sizes = [[700, 700], [900, 900], [1100, 1100]] # 此处数字代表像素点大小
img_path = "solar_panel/val/val_img"
label_path = "solar_panel/val/val_label"
# split_image_v1为平切算法，即切割图片无重叠
split_image_v1.split_images_segment_v1(img_path, split_sizes, label_path, with_edge=False) # with_edge: 是否保留在边缘的目标
# split_image_v2为有重叠切法，即切割图片之前有重叠部分
# split_image_v2.split_images_segment_v2(img_path, split_sizes, label_path, with_edge=False) # with_edge: 是否保留在边缘的目标


Image segmentation completed!


运行以下代码，打印出图片看看标注是否正确。确定正确后删除images_with_contour文件夹

In [5]:
img_path = "solar_panel/train/images"
label_path = "solar_panel/train/labels"
yolo_dataset_utils.show_batch_image_segment(img_path, label_path)

已生成图片保存至solar_panel/train/images_with_contour\20150115_1100_1100_1_10_contour.png
已生成图片保存至solar_panel/train/images_with_contour\20150115_1100_1100_2_10_contour.png
已生成图片保存至solar_panel/train/images_with_contour\20150115_1100_1100_3_3_contour.png
已生成图片保存至solar_panel/train/images_with_contour\20150115_1100_1100_3_4_contour.png
已生成图片保存至solar_panel/train/images_with_contour\20150115_1100_1100_3_5_contour.png
已生成图片保存至solar_panel/train/images_with_contour\20150115_1100_1100_4_2_contour.png
已生成图片保存至solar_panel/train/images_with_contour\20150115_1100_1100_4_3_contour.png
已生成图片保存至solar_panel/train/images_with_contour\20150115_1100_1100_4_4_contour.png
已生成图片保存至solar_panel/train/images_with_contour\20150115_1100_1100_4_5_contour.png
已生成图片保存至solar_panel/train/images_with_contour\20150115_1100_1100_5_3_contour.png
已生成图片保存至solar_panel/train/images_with_contour\20150115_1100_1100_5_4_contour.png
已生成图片保存至solar_panel/train/images_with_contour\20150115_1100_1100_5_5_contour.png
已生成图片保存至solar_panel/train/


KeyboardInterrupt



至此，数据集准备工作完毕。

## 2.模型训练
模型训练通过命令行执行。需要配置一个.yaml文件，说明数据集的位置。yaml文件范例见ultralytics/training/config



config配置完成后，在命令行运行如下代码开启训练过程：
*yolo task=segment mode=train project=solar_panel name=solar_panel model=ultralytics/training/pre_models/yolov8x-seg.pt data=ultralytics/training/config/solar_panel.yaml  batch=16  epochs=401  device=0  patience=50  save_period=50 degrees=45 flipud=0.5 fliplr=0.5*

其中，degrees，flipud和fliplr为数据增强的参数。更多参数详见官网：[https://docs.ultralytics.com/usage/cfg/#export](https://docs.ultralytics.com/usage/cfg/#export)

## 3.模型预测
模型预测代码示例如下：

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.gif', '.webp')

def get_config():
    config_path = "D:/Code/gitcode/train_pipeline/ultralytics/inference/config/solarpanel_config.yaml"
    config_file = open(config_path, 'r', encoding='utf-8')
    file_info = config_file.read()
    config_dict = yaml.safe_load(file_info)
    return config_dict

def pre_handle(config_dict):
    image_path = config_dict['image_path']
    split_arr = config_dict['split_arr']
    coordinates = config_dict['coordinates']
    pixel_arr = yolov8_seg_handle.lon_lat_to_pixel(image_path, coordinates)
    mid_dict = {'pixel_arr': pixel_arr}  # 用于存储中间结果
    # 图片切割
    logging.info('开始切割图片')
    split_images_dict = yolov8_seg_handle.split_image_large(image_path, split_arr, pixel_arr)
    mid_dict['split_images_dict'] = split_images_dict
    logging.info("切割图片完成!!!")
    return mid_dict

def single_predict(image_path, config_dict):
    config_dict['image_path'] = image_path
    # 2，预处理
    mid_dict = pre_handle(config_dict)
    # 3，模型识别
    mid_dict = yolov8_seg_handle.model_predict(config_dict, mid_dict)
    # 4，后处理
    mid_dict = yolov8_seg_handle.after_handle_merge_polygon(config_dict, mid_dict)

    # 5，生成输出结果
    mid_dict = yolov8_seg_handle.create_geojson(config_dict, mid_dict)
    # 6，输出图片
    yolov8_seg_handle.show_images(config_dict, mid_dict)

def batch_predict(image_file_path, config_dict):
    image_names = os.listdir(image_file_path)
    for image_name in image_names:
        if image_name.endswith(IMAGE_EXTENSIONS):
            # if image_name != 'car20210205.tif':
            #    continue
            print('预测，', image_name)
            image_path = image_file_path + '/' + image_name
            single_predict(image_path, config_dict)
def main():
    start_time = time.time()
    # 1，参数处理
    config_dict = get_config()
    # image_path = sys.argv[1]
    # coordinates = sys.argv[2]
    image_path = "D:/Code/gitcode/train_pipeline/examples/yolov8_solar_panel/solar_panel/images/20210313.tif"
    coordinates = '[]' #'[[119.64718, 34.44381], [119.68986, 34.41249]]'
    config_dict['out_flag'] = False
    config_dict['start_time'] = start_time
    config_dict['image_path'] = image_path
    # config_dict['image_path'] = image_path
    coordinates = json.loads(coordinates)
    config_dict['coordinates'] = coordinates
    # 预测一张图片
    single_predict(image_path, config_dict)

    image_file_path = "/data/Solar_Panel/solarpanel_test"
    # 预测多张图片
    # batch_predict(image_file_path, config_dict)



if __name__ == '__main__':
    main()



INFO:root:开始切割图片
INFO:root:切割图片完成!!!
INFO:root:开始模型预测
  0%|          | 0/9 [00:00<?, ?it/s]
0: 640x640 2 SolarPanels, 1: 640x640 2 SolarPanels, 2: 640x640 4 SolarPanels, 3: 640x640 2 SolarPanels, 4: 640x640 4 SolarPanels, 5: 640x640 1 SolarPanel, 6: 640x640 4 SolarPanels, 7: 640x640 7 SolarPanels, 8: 640x640 4 SolarPanels, 9: 640x640 8 SolarPanels, 10: 640x640 2 SolarPanels, 11: 640x640 2 SolarPanels, 12: 640x640 (no detections), 13: 640x640 (no detections), 14: 640x640 (no detections), 15: 640x640 (no detections), 1283.6ms
Speed: 2.5ms preprocess, 80.2ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
 11%|█         | 1/9 [00:02<00:18,  2.27s/it]
0: 640x640 (no detections), 1: 640x640 (no detections), 2: 640x640 (no detections), 3: 640x640 (no detections), 4: 640x640 (no detections), 5: 640x640 (no detections), 6: 640x640 (no detections), 7: 640x640 (no detections), 8: 640x640 6 SolarPanels, 9: 640x640 2 SolarPanels, 10: 640x640 11 SolarPanels, 11: 640x640 10 SolarPan

D:/Code/gitcode/train_pipeline/examples/yolov8_solar_panel/solar_panel/images/out/20210313_show\20210313_contour_best.tif


INFO:root:生成图片完成
